## Introduction
Because Pittsburgh is always called a food city, we decided to base our metric on density of food per Allegheny county zip code. By "food", we mean places that residents can buy food, like supermarkets, convenience stores, restaurants, and farmers markets. Combined, these places will give us a good idea of how much food there actually is per zip code. Some other options we explored included sorting dog names by neighborhood, health quality per zip code, and food quality per zip code.

## Metric
Our data sets are the following: 
* [Allegheny County Supermarkets and Convenience Stores](https://data.wprdc.org/dataset/allegheny-county-supermarkets-convenience-stores/resource/626357fa-c95d-465f-9a02-3121655b2b78)
* [Allegheny County Farmers Markets](https://data.wprdc.org/dataset/allegheny-county-farmers-markets-locations)
* [Allegheny County Restaurant Food Facility Inspections and Locations](https://data.wprdc.org/dataset/allegheny-county-restaurant-food-facility-inspection-violations)
Our metric is the amount of food facilities per zip code. We chose this specifically because all three of these data sets have a zip code listing. This makes it easier to combine all three of these datasets into one, as they all have a common column.  

In [2]:
import pandas as pd
import numpy as np